In [1]:
# default_exp learner

In [2]:
import nbdev.showdoc as literacy

In [3]:
#export
from speechsep.imports import *
from speechsep.data import *
from speechsep.core import *

/home/cerberus/.anaconda3/envs/fai2/lib/python3.7/site-packages/fastcore/basics.py:790: UserWarning: `patch_property` is deprecated and will be removed; use `patch(as_prop=True)` instead
  warnings.warn("`patch_property` is deprecated and will be removed; use `patch(as_prop=True)` instead")


# Learner

## Prep

In [4]:
data = DataBlock(blocks=(AudioBlock(), CategoryBlock),
                  get_items=get_audio_files,
                  splitter=RandomSplitter(),
                  get_y=parent_label)

fn = Path("/home/cerberus/Music/LibriSpeech/test-clean/")
dls = data.dataloaders(fn, item_tfms=(Resample(22050), Clip(5), Spectify(), PhaseManager()), bs=4)

dls.show_batch()

TypeError: 'NoneType' object is not iterable

In [ ]:
from torch.nn import Sequential, Conv2d, ReLU, BatchNorm2d, MaxPool2d, Linear
from fastai2.learner import Learner
from fastai2.metrics import accuracy

## Model Audio

In [ ]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(2, 4, kernel_size=2, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(54272, 2)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

spec_model = Net()

In [ ]:
learner = Learner(dls, resnet34t, metrics=error_rate)

In [ ]:
learner.fit_one_cycle(4)